In [7]:
import networkx as nx
import matplotlib.pyplot as plt
import json
import time
import datetime

In [2]:
def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f) #, default={'sender': 'source'})
        _attrs = dict(source='sender', target='receiver', name='guid',
              key='guid', link='links')
    #return nx.readwrite.node_link_graph(js_graph, {'link': 'links', 'source': 'sender', 'target': 'receiver', 'key': 'guid'})
    return nx.readwrite.node_link_graph(js_graph, directed=True, multigraph=False, attrs={'link': 'links', 'source': 'sender', 'target': 'receiver', 'key': 'guid', 'name': 'guid'} )

G = read_json_file("data.json")
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 6489
Number of edges: 52318
Average in degree:   8.0626
Average out degree:   8.0626


In [3]:
print ("SCC: ", nx.number_strongly_connected_components(G))
print ("WCC: ", nx.number_weakly_connected_components(G))

SCC:  6007
WCC:  107


In [4]:
G_ud = G.to_undirected()
print("is_connected", nx.is_connected(G_ud))
print("is_strongly_connected", nx.is_strongly_connected(G))
print("number_connected_components", nx.number_connected_components(G_ud))

G_mc = list(G_ud.subgraph(c).copy() for c in nx.connected_components(G_ud))
# Connected components are sorted in descending order of their size
G_components = list(nx.connected_component_subgraphs(G_ud))
_components = (G_ud.subgraph(c) for c in nx.connected_components(G_ud))

# todo: do for-loop to run algorithm for all components
G_mc = G_components[0]

print("is_connected", nx.is_connected(G_mc)) 
print("number_connected_components", nx.number_connected_components(G_mc))
print("dencity", nx.density(G_mc))
print(nx.info(G_mc))

is_connected False
is_strongly_connected False
number_connected_components 107
is_connected True
number_connected_components 1
dencity 0.003164320541396733
Name: 
Type: Graph
Number of nodes: 5330
Number of edges: 44939
Average degree:  16.8627


In [6]:
# Betweenness centrality 
current_time = time.time()
bet_cent_edges = nx.edge_betweenness_centrality(G_mc, weight='lastTs')
 
print("bet_cent_edges took", time.time()-current_time)

print(len(bet_cent_edges))
print(list(bet_cent_edges)[0])
print(bet_cent_edges[list(bet_cent_edges)[0]])

bet_cent_edges took 1273.4692180156708
44939
('4058a3f97f22afe22f4a7f54fe2523af', '2415908ec83ea1597814d4bead9aacec')
3.5206841956838524e-07


In [8]:
print("bet_cent_edges took", str(datetime.timedelta(seconds=time.time()-current_time))) 

bet_cent_edges took 0:25:19.075013


In [ ]:
edges_max_goal = 1000
G_reduced = G_mc.copy()
print(nx.info(G_reduced))
removed_edges = []

def remove_edges(items):
    sorted_bet_cent_edges = sorted(items,reverse=False, key=lambda x: x[1])
    
    print("count :", len(list(sorted_bet_cent_edges)))

    for bet_cent in sorted_bet_cent_edges: 
        if G_reduced.degree(bet_cent[0]) > 2 and G_reduced.degree(bet_cent[1]) > 2:
            G_reduced.remove_edge(*bet_cent)  
            removed_edges.append(bet_cent)
        
        if (G_reduced.number_of_edges() < edges_max_goal):
            print("done :",G_reduced.number_of_edges())
            break
             
print("G_reduced edges before:", G_reduced.number_of_edges()) 
current_time = time.time()
remove_edges(bet_cent_edges)
print("removing_edges took", time.time()-current_time)
print("G_reduced edges after:", G_reduced.number_of_edges())  
print("removed_edges:", len(removed_edges))

 

In [ ]:
def postprocess(items):
    removed = sorted(items, reverse=True) 
    _components = list(nx.connected_component_subgraphs(G_reduced)) 
    
    for edge in removed: 
        if nx.number_connected_components(G_reduced) == 1:
            print("break")
            break
            
        for c in _components:
            if c.has_node(edge[0]) and c.has_node(edge[1]):
                break
            elif c.has_node(edge[0]) or c.has_node(edge[1]): 
                G_reduced.add_edge(*edge)
                _components = list(nx.connected_component_subgraphs(G_reduced))
    
current_time = time.time()
postprocess(removed_edges)
 
print("postprocessing took:", time.time()-current_time)  
print(nx.info(G_reduced))

In [ ]:
print("number_connected_components", nx.number_connected_components(G_mc))
print("number_connected_components", nx.number_connected_components(G_reduced))

In [ ]:
# save reduced graph into a file
_attrs = dict(source='sender', target='receiver', name='guid',
              key='guid', link='links')
s2 = nx.readwrite.node_link_data(G_reduced, attrs={'link': 'links', 'source': 'sender', 'target': 'receiver', 'key': 'guid', 'name': 'guid'})

with open('data_processed.json', 'w') as outfile:
    json.dump(s2, outfile)

